In [1]:
import ee
import geemap
import math
import itertools
import pandas as pd

In [2]:
#Initialize Google Earth Engine
#ee.Authenticate() just needed the 1st time
ee.Initialize()

In [3]:
#Data loads

#loads feature collection data from Google Earth Engine - We can also upload other feature collections
counties = ee.FeatureCollection("TIGER/2018/Counties")

#filter LA County
la_county = counties.filter(ee.Filter.eq('NAME', 'Los Angeles'))

In [4]:
#loads cities data - Not in current use

la_county_income_zipcode2 = ee.FeatureCollection("projects/california-lawn-detection/assets/income_zipcode2019")
la_county_income_zipcode = la_county_income_zipcode2.select(ee.List(['zipcode', '2019zipcod','shape_area']), ee.List(['ZipCode', 'Median_Income','Area_sqft']))

In [5]:
image_collection_2020_modis = (
    ee.ImageCollection('MODIS/006/MOD11A2')
    .filterDate('2020-04-01', '2020-06-30')
    .select(['LST_Day_1km', 'LST_Night_1km'])
)

image_2020_modis_clipped = image_2020_modis.clip(la_county)

NameError: name 'image_2020_modis' is not defined

In [46]:
z90802 = la_county_income_zipcode.filter(ee.Filter.eq('ZipCode', '90802')) 

In [56]:
year_list = [2010, 2012, 2014, 2016, 2018, 2020]
zipcode_list = ['90802','90732','90744']

In [107]:
def get_images(param_dict):
    source_image_collection = params['source_image_collection']
    years = param_dict['years']
    counties = param_dict['counties']

    image_names = []
    images = []

    combos = list(itertools.product(years, counties.keys()))
    for i in combos:
        year = str(i[0])
        county = i[1]

        image_name = str(i[0])+'_'+i[1]
        image_names.append(image_name)

        image = ee.ImageCollection(source_image_collection)\
                                .filterDate(f'{year}-04-01', f'{year}-06-30')\
                                .select(['LST_Day_1km', 'LST_Night_1km'])\
                                .median().clip(counties[county])
        images.append(image)
           
    return dict(zip(image_names, images))        


In [108]:
params = {
        'source_image_collection' : 'MODIS/006/MOD11A2',
        'years' : [2010,2012,2014,2016,2018,2020],
        'counties': {'la_county': la_county}
         }

images = get_images(params)
images

{'2010_la_county': <ee.image.Image at 0x1a5457b6d00>,
 '2012_la_county': <ee.image.Image at 0x1a5456aa310>,
 '2014_la_county': <ee.image.Image at 0x1a5457d12e0>,
 '2016_la_county': <ee.image.Image at 0x1a5457d1880>,
 '2018_la_county': <ee.image.Image at 0x1a5457d1e20>,
 '2020_la_county': <ee.image.Image at 0x1a5457db3d0>}

In [114]:
def temperature_calculation(modis_image, shape, pixel_scale):
    
    reducers = ee.Reducer.mean().combine(ee.Reducer.min(),sharedInputs=True).combine(ee.Reducer.max(),sharedInputs=True)
    
    mean_temperature = modis_image.reduceRegion(
        reducer = reducers,
        geometry = shape.geometry(), 
        scale = pixel_scale, 
        maxPixels = 1e13)
    
    lst_day_mean = mean_temperature.getInfo().get('LST_Day_1km_mean') * .02 - 273
    lst_day_max = mean_temperature.getInfo().get('LST_Day_1km_max') * .02 - 273
    lst_day_min = mean_temperature.getInfo().get('LST_Day_1km_min') * .02 - 273
    lst_night_mean = mean_temperature.getInfo().get('LST_Night_1km_mean') * .02 - 273
    lst_night_max = mean_temperature.getInfo().get('LST_Night_1km_max') * .02 - 273
    lst_night_min = mean_temperature.getInfo().get('LST_Night_1km_min') * .02 - 273
    
    return lst_day_mean, lst_day_max, lst_day_min, lst_night_mean, lst_night_max, lst_night_min

In [ ]:
temperature_calculation(image_2020_modis_clipped, shape, pixel_scale)

In [121]:
year_list = [2010, 2012, 2014, 2016, 2018, 2020]
zipcode_list = ['90802','90732','90744']

In [133]:
keys = {'year','polygon',
        'lst_day_mean','lst_day_max', 'lst_day_min', 
        'lst_night_mean', 'lst_night_max','lst_night_min'}

year_list = []
polygon_list = []
lst_day_mean = []
lst_day_max = []
lst_day_min = []
lst_night_mean = []
lst_night_max = []
lst_night_min = []
    
    
for i in zipcode_list:
    for j in list(images.items()):
        im = j[1]
        name = j[0]
        
        polygon_list.append(i)  
        year_list.append(j[0][:4])  
        
        #start = time()
        polygon = la_county_income_zipcode.filter(ee.Filter.eq('ZipCode', i))
        temperature = temperature_calculation(im, polygon, 1000)
        
        temperature = list(temperature)
        lst_day_mean.append(temperature[0]) 
        lst_day_max.append(temperature[1])
        lst_day_min.append(temperature[2])
        lst_night_mean.append(temperature[3])
        lst_night_max.append(temperature[4])
        lst_night_min.append(temperature[5])
        
        #end = time()
        
        #print(f'Zip Code: {i}, Year: {j[0][:4]} ::: completed in {end-start} seconds.')
              

In [135]:
df_list = list(zip(polygon_list, year_list,lst_day_mean,lst_day_max,lst_day_min,lst_night_mean,lst_night_max,lst_night_min))

In [138]:
df = pd.DataFrame(df_list, columns = ['polygon','year',
        'lst_day_mean','lst_day_max', 'lst_day_min', 
        'lst_night_mean', 'lst_night_max','lst_night_min'])

In [139]:
df

,polygon,year,lst_day_mean,lst_day_max,lst_day_min,lst_night_mean,lst_night_max,lst_night_min
0,90802,2010,31.420895,34.80,22.28,15.106607,15.69,14.42
1,90802,2012,32.532479,35.74,27.22,16.591078,17.88,15.40
2,90802,2014,33.924403,37.76,26.89,17.918137,19.08,16.90
3,90802,2016,32.144044,35.30,23.31,15.546807,16.68,14.70
4,90802,2018,32.212561,35.56,25.52,15.085133,15.98,13.10
5,90802,2020,33.498712,36.60,32.20,16.983264,18.17,14.60
6,90732,2010,30.168950,35.28,24.18,13.365220,14.38,12.34
7,90732,2012,31.716974,35.70,26.61,14.965129,15.98,14.14
8,90732,2014,34.468693,38.94,28.56,15.917846,16.86,15.10
9,90732,2016,31.913846,35.12,29.07,14.036610,15.30,13.02
